In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My\ Drive/NLP/cnn-text-classification-pytorch/

/content/drive/My Drive/NLP/cnn-text-classification-pytorch


In [0]:
ls

custom_mapping.pickle  model.py                        rt-polaritydata.tar
LICENSE                mydatasets.py                   snapshot/
main.py                __pycache__/                    train.py
mapping.pickle         README.md                       try1/
model_glove_2.pth      rt-polaritydata/                val_glove.pickle
model_glove.pth        rt-polaritydata.README.1.0.txt  val_set.pickle


In [0]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
import numpy as np
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

np.random.seed(500)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
path = "../data/Subtask-1-master/train.csv"

In [0]:
import pandas as pd

corpus = pd.read_csv(path, encoding='utf-8')
corpus['sentence'].dropna(inplace=True)
corpus['sentence'] = [sent.lower() for sent in corpus['sentence']]
corpus['sentence'] = [word_tokenize(word) for word in corpus['sentence']]

In [0]:
corpus['sentence']

0        [goodfellow, 's, theory, has, been, questioned...
1        [however, ,, both, campaigners, and, pro-peopl...
2        [things, could, have, been, even, better, if, ...
3        [the, new, request, ,, if, approved, ,, would,...
4        [companies, in, financial, difficulty, can, cu...
                               ...                        
12995    [the, department, said, it, conducts, thorough...
12996    [the, mccain, episode, may, sound, largely, ha...
12997    [merkel, said, in, a, speech, to, her, conserv...
12998    [``, ``, the, entire, drug, scene, has, change...
12999    [in, 2009, ,, the, group, said, adults, should...
Name: sentence, Length: 13000, dtype: object

In [0]:
# glove

w2v = []
w2i = {}
words = []

idx = 0
with open('../data/glove.6B.300d.txt','rb') as f:
  for l in f:
    line = l.decode().split()
    word =  line[0]
    words.append(word)
    w2i[word] = idx
    idx +=1
    vect = np.array(line[1:]).astype(np.float)
    w2v.append(vect)



In [0]:
for i in range(len(corpus['sentence'])):
  sent = corpus['sentence'][i]
  for j in range(len(sent)):
    if '...' in sent[j]:
      corpus['sentence'][i][j] = sent[j].replace('...','')
    if '..' in sent[j]:
      corpus['sentence'][i][j] = sent[j].replace('..','')


In [0]:
for i in range(len(corpus['sentence'])):
  sent = corpus['sentence'][i]
  for j in range(len(sent)):
    if '-' in sent[j]:
      x = sent[j].split('-')
      corpus['sentence'][i][j] = x[0]
      if j+1 == len(sent):
        corpus['sentence'][i].append(x[1])
      else:
        corpus['sentence'][i].insert(j+1,x[1])

In [0]:
for i in range(len(corpus['sentence'])):
  sent = corpus['sentence'][i]
  for j in range(len(sent)):
    if '/' in sent[j]:
      x = sent[j].split('/')
      corpus['sentence'][i][j] = x[0]
      if len(x)>1:
        corpus['sentence'][i].insert(j+1,x[1])

In [0]:
for i in range(len(corpus['sentence'])):
  sent = corpus['sentence'][i]
  for j in range(len(sent)):
    if "'" in sent[j]:
      if sent[j] not in words:
        corpus['sentence'][i][j] = sent[j].replace("'","")
      # x = sent[j].split('-')
      # corpus['sentence'][i][j] = x[0]
      # corpus['sentence'][i].insert(j+1,x[1])

In [0]:
for i in range(len(corpus['sentence'])):
  sent = corpus['sentence'][i]
  for j in range(len(sent)):
    if "." in sent[j]:
      if sent[j] not in words:
        if len(sent[j].split('.')) > 2 or len(sent[j].split('.')) == 1:
          corpus['sentence'][i][j] = sent[j].replace(".","")
        else:
          if len(sent[j].split('.')) == 2:
            x = sent[j].split('.')
            corpus['sentence'][i][j] = x[0]
            corpus['sentence'][i].insert(j+1, x[1])
            # print(corpus['sentence'][i][j], corpus['sentence'][i][j+1], '\n')

In [0]:
not_in = []
for sent in corpus['sentence']:
  for word in sent:
    if word not in words:
      if word not in not_in:
        not_in.append(word)
        # print(word)

In [0]:
len(not_in)

1381

1381

In [0]:
print(not_in)

['kingdomof', '', 'forwardnot', 'stakethe', 'nonemergencies', 'circumastance', 'r-nc', 'reynaga', 'irisin', 'cuntry', 'shkreli', 'thatdespite', 'forcehe', 'ntgr', 'bfsr', 'ceramides', 'otillar', 'scratchpad', 'mcritchie', 'enria', 'watchwordalthough', 'mosychuk', 'aedpa', 'dipierro', 'panelo', 'arconic', 'paynet', 'gorsuchs', 'suicidenow', 'kyrsten', 'freiborg', '5bn', 'depomed', 'amsellem', 'overperformances', 'thcb', 'privitera', 'legimately', 'burkan', 'zonszein', '10,554', 'brochez', 'pfapa', 'bingener', 'entresto', 'candidateand', 'zarxio', 'tdn2k', 'confounders', 'overcounted', 'rebirthed', 'altmaier', 'barcap', 'shithole', 'lignos', 'sciple', 'morens', 'reclosing', 'roadfor', 'unpeg', 'currencythe', 'somepart', 'allthat', 'staehr', 'baumblit', 'khorsand', 'innereye', 'topkill', '1+1', 'kilimnik', 'fralick', 'krautzberger', 'truog', 'luján', 'screenos', 'unbooked', 'ohions', 'sowanick', 'neurobiologically', 'killino', 'incel', 'nn27303398', 'rcep', 'piampongsant', 'oubina', 'non

In [0]:
corpus_words = []
for sent in corpus['sentence']:
  for word in sent:
    if word not in not_in:
      if word not in corpus_words:
        corpus_words.append(word)

In [0]:
mapping = []
custom_mapping = {0 : 0}
idx = 1
for w in corpus_words:
  mapping.append(w2v[w2i[w]])
  custom_mapping[w] = idx
  idx+=1

pad = [0]*300
mapping.insert(0,pad)

mapping = np.array(mapping)

for i in range(len(not_in)):
  rnd = np.array([np.random.uniform(-1,1,300)])
  mapping = np.append(mapping,rnd,axis = 0)
  custom_mapping[not_in[i]] = idx
  idx+=1


In [0]:
import pickle

with open("custom_mapping.pickle",'wb') as f:
  pickle.dump(custom_mapping, f)

In [0]:
type(mapping[0])

numpy.ndarray

In [0]:
len(mapping) == len(corpus_words) + len(not_in) + 1

True

In [0]:
max_len = 0
this = 0
for x in corpus['sentence']:
  if len(x)>max_len:
    max_len = len(x)

In [0]:
data = []
label = []
idx = 0
padding = 2
for sent in corpus['sentence']:
  s = [0]*padding
  for word in sent:
    s.append(custom_mapping[word])
  while(len(s) < max_len + 2*padding):
    s.append(0)
  data.append(s)
  label.append(corpus['gold_label'][idx])
  idx+=1

In [0]:
data = np.array(data)
label = np.array(label)

In [0]:
len(data[907])

593

593

In [0]:
data_indices = []
for i in range(len(label)):
  data_indices.append(i)

In [0]:
import torch.nn as nn
import torch

In [0]:
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
seed = 2374

In [0]:
from sklearn.model_selection import train_test_split

split = 0.2
splits = train_test_split(data_indices, test_size = split, random_state = seed, stratify = label)

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

train_sampler = SubsetRandomSampler(splits[0])
valid_sampler = SubsetRandomSampler(splits[1])

In [0]:
class text_dataset(torch.utils.data.Dataset):
    
    def __init__(self, data, label):
        self.data = data
        self.labels = label

        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, idx):

        l = torch.tensor(self.labels[idx]).long()
        sentence = torch.tensor(self.data[idx]).long()

        return sentence, l



In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class CNN_Text(nn.Module):        

    def __init__(self, args, weight_matrix, train_weights=False):
        super(CNN_Text, self).__init__()
        self.args = args
        self.embed, num_embeddings, embedding_dim = self.create_emb_layer(weight_matrix, train_weights)

        V = num_embeddings
        D = embedding_dim
        C = args.class_num
        Ci = 1
        Co = args.kernel_num
        Ks = args.kernel_sizes

        # self.convs1 = [nn.Conv2d(Ci, Co, (K, D)) for K in Ks]
        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])
        '''
        self.conv13 = nn.Conv2d(Ci, Co, (3, D))
        self.conv14 = nn.Conv2d(Ci, Co, (4, D))
        self.conv15 = nn.Conv2d(Ci, Co, (5, D))
        '''
        self.dropout = nn.Dropout(args.dropout)
        self.fc1 = nn.Linear(len(Ks)*Co, C)
    
    def create_emb_layer(self, weight_matrix, train_weights=False):
        num_embeddings, embedding_dim = weight_matrix.size()
        emb_layer = nn.Embedding(num_embeddings, embedding_dim)
        emb_layer.load_state_dict({'weight': weight_matrix})
        if train_weights:
            emb_layer.weight.requires_grad = True
        else:
            emb_layer.weight.requires_grad = False

        return emb_layer, num_embeddings, embedding_dim

    def conv_and_pool(self, x, conv):
        x = F.relu(conv(x)).squeeze(3)  # (N, Co, W)
        x = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x

    def forward(self, x):
        x = self.embed(x)  # (N, W, D)
        # print(x.size())
        
        if self.args.static:
            x = Variable(x)

        x = x.unsqueeze(1)  # (N, Ci, W, D)

        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)

        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)

        x = torch.cat(x, 1)

        '''
        x1 = self.conv_and_pool(x,self.conv13) #(N,Co)
        x2 = self.conv_and_pool(x,self.conv14) #(N,Co)
        x3 = self.conv_and_pool(x,self.conv15) #(N,Co)
        x = torch.cat((x1, x2, x3), 1) # (N,len(Ks)*Co)
        '''
        x = self.dropout(x)  # (N, len(Ks)*Co)
        logit = self.fc1(x)  # (N, C)
        return logit


In [0]:
import os
import sys
import torch
import torch.autograd as autograd
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


def train(train_iter, dev_iter, model, args):
    if args.cuda:
        model.cuda()
        print("On Cuda")

    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=12, gamma=0.1)
    weights = [1,10]
    weights = torch.FloatTensor(weights).cuda()
    criterion = nn.CrossEntropyLoss(weight=weights)

    steps = 0
    best_acc = 0
    last_step = 0
    to_return = {}
    max_so_far = 0.0
    for epoch in range(1, args.epochs+1):
        model.train()
        train_preds = None
        train_labels = None
        running_loss = 0.0
        total_loss = 0.0
        mini_batch = 0
        print("epoch: ",epoch)
        scheduler.step()

        for feature, labels in train_iter:
            mini_batch+=1
            
            if args.cuda:
                feature, labels = feature.cuda(), labels.cuda()

            optimizer.zero_grad()
            outputs = model(feature)

            # print('outputs size', outputs.size())
            # print('target vector', target.size())
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            total_loss += loss.item()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            steps += 1
            if train_preds is None or train_labels is None:
              train_preds = np.argmax(outputs.detach().cpu().numpy(), axis=1).flatten()
              train_labels = labels.cpu().numpy().flatten()
            else:
              train_preds = np.concatenate((train_preds, np.argmax(outputs.detach().cpu().numpy(), axis=1).flatten()))
              train_labels = np.concatenate((train_labels, labels.cpu().numpy().flatten()))


            loss.backward()
            optimizer.step()
            # scheduler.step()

            if mini_batch % 100 == 0:    # print every 100 mini-batches
              print('[%d, %5d] loss: %.3f' % (epoch , mini_batch , running_loss / 100))
              running_loss = 0.0
        
        print("Training loss in epoch %d is %.3f" % (epoch , total_loss / len(train_loader)))
        print("Training accuracy in epoch %d is %.3f" % (epoch, accuracy_score(train_labels, train_preds) * 100))
        print("Training precision in epoch %d is %.3f" % (epoch , precision_score(train_labels, train_preds) * 100))
        print("Training recall in epoch %d is %.3f" % (epoch , recall_score(train_labels, train_preds) * 100))
        print("Training F1-score in epoch %d is %.3f" % (epoch, f1_score(train_labels, train_preds) * 100))
        
        eval(dev_iter, model, criterion, epoch, args, to_return, max_so_far)
        print()
        print("-"*90)
        print()
        
    return to_return


def eval(data_iter, model, criterion, epoch, args, to_return, max_so_far):
    
    model.eval()
    test_loss = 0.0
    test_preds = None
    test_labels = None
    test_inputs = None
    # corrects, avg_loss = 0, 0
    with torch.no_grad():

        for data in data_iter:
            inputs, labels = data
            if args.cuda:
                inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
          
            test_loss += loss.item()
            if test_preds is None or test_labels is None:
                test_preds = np.argmax(outputs.detach().cpu().numpy(), axis=1).flatten()
                test_labels = labels.cpu().numpy().flatten()
                test_inputs = inputs.cpu().numpy()
            else:
                test_preds = np.concatenate((test_preds, np.argmax(outputs.detach().cpu().numpy(), axis=1).flatten()))
                test_labels = np.concatenate((test_labels, labels.cpu().numpy().flatten()))
                test_inputs = np.concatenate((test_inputs, inputs.cpu().numpy()), axis = 0)

            # test_accuracy += flat_accuracy(outputs[0], labels)
    
    print("Test loss in epoch %d is %.3f" % (epoch, test_loss / len(val_loader)))
    print("Test accuracy in epoch %d is %.3f" % (epoch, accuracy_score(test_labels, test_preds) * 100))
    print("Test precision in epoch %d is %.3f" % (epoch, precision_score(test_labels, test_preds) * 100))
    print("Test recall in epoch %d is %.3f" % (epoch, recall_score(test_labels, test_preds) * 100))
    print("Test F1-score in epoch %d is %.3f" % (epoch, f1_score(test_labels, test_preds) * 100))
    if f1_score(test_labels, test_preds)*100 > max_so_far:
      max_so_far = f1_score(test_labels, test_preds)*100
      to_return['pred'] = test_preds
      to_return['label'] = test_labels
      to_return['inputs'] = test_inputs



def predict(text, model, text_field, label_feild, cuda_flag):
    assert isinstance(text, str)
    model.eval()
    # text = text_field.tokenize(text)
    text = text_field.preprocess(text)
    text = [[text_field.vocab.stoi[x] for x in text]]
    x = torch.tensor(text)
    x = autograd.Variable(x)
    if cuda_flag:
        x = x.cuda()
    print(x)
    output = model(x)
    _, predicted = torch.max(output, 1)
    #return label_feild.vocab.itos[predicted.data[0][0]+1]
    return label_feild.vocab.itos[predicted.data[0]+1]


def save(model, save_dir, save_prefix, steps):
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    save_prefix = os.path.join(save_dir, save_prefix)
    save_path = '{}_steps_{}.pt'.format(save_prefix, steps)
    torch.save(model.state_dict(), save_path)


In [0]:
weight_matrix = torch.tensor(mapping)

In [0]:
import os
import argparse
import datetime
import torch


parser = argparse.ArgumentParser(description='CNN text classificer')
# learning
parser.add_argument('-lr', type=float, default=0.001, help='initial learning rate [default: 0.001]')
parser.add_argument('-epochs', type=int, default=30, help='number of epochs for train [default: 256]')
parser.add_argument('-batch-size', type=int, default=64, help='batch size for training [default: 64]')
parser.add_argument('-log-interval',  type=int, default=1,   help='how many steps to wait before logging training status [default: 1]')
parser.add_argument('-test-interval', type=int, default=100, help='how many steps to wait before testing [default: 100]')
parser.add_argument('-save-interval', type=int, default=500, help='how many steps to wait before saving [default:500]')
parser.add_argument('-save-dir', type=str, default='try1', help='where to save the snapshot')
parser.add_argument('-early-stop', type=int, default=1000, help='iteration numbers to stop without performance increasing')
parser.add_argument('-save-best', type=bool, default=True, help='whether to save when get best performance')
# data 
parser.add_argument('-shuffle', action='store_true', default=True, help='shuffle the data every epoch')
# model
parser.add_argument('-dropout', type=float, default=0.5, help='the probability for dropout [default: 0.5]')
parser.add_argument('-max-norm', type=float, default=3.0, help='l2 constraint of parameters [default: 3.0]')
parser.add_argument('-embed-dim''''  ''', type=int, default=300, help='number of embedding dimension [default: 128]')
parser.add_argument('-kernel-num', type=int, default=100, help='number of each kind of kernel')
parser.add_argument('-kernel-sizes', type=str, default='3,4,5', help='comma-separated kernel size to use for convolution')
parser.add_argument('-static', action='store_true', default=True, help='fix the embedding')
# device
parser.add_argument('-device', type=int, default=-1, help='device to use for iterate data, -1 mean cpu [default: -1]')
parser.add_argument('-no-cuda', action='store_true', default=False, help='disable the gpu')
# option
parser.add_argument('-snapshot', type=str, default=None, help='filename of model snapshot [default: None]')
parser.add_argument('-predict', type=str, default=None, help='predict the sentence given')
parser.add_argument('-test', action='store_true', default=False, help='train or test')
args = parser.parse_args(args=[])


# load data 
print("\nLoading data...")
# text_field = data.Field(lower=True)
# label_field = data.Field(sequential=False)
# train_iter, dev_iter = mr(text_field, label_field, device=-1, repeat=False)
dataset = text_dataset(data, label)
train_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=args.batch_size,
                                           sampler = train_sampler)
val_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=args.batch_size,
                                           sampler = valid_sampler)

# train_iter, dev_iter, test_iter = sst(text_field, label_field, device=-1, repeat=False)


# update args and print
args.embed_num = weight_matrix.size()[1]
args.class_num = 2 #binary
args.cuda = (not args.no_cuda) and torch.cuda.is_available(); del args.no_cuda
args.kernel_sizes = [int(k) for k in args.kernel_sizes.split(',')]
args.save_dir = os.path.join(args.save_dir, datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

print("\nParameters:")
for attr, value in sorted(args.__dict__.items()):
    print("\t{}={}".format(attr.upper(), value))
print("Training sentences: ", int(len(dataset)*(1-split)))
print("Validation sentences: ", int(len(dataset)*(split)))



Loading data...

Parameters:
	BATCH_SIZE=64
	CLASS_NUM=2
	CUDA=True
	DEVICE=-1
	DROPOUT=0.5
	EARLY_STOP=1000
	EMBED_DIM  =300
	EMBED_NUM=300
	EPOCHS=30
	KERNEL_NUM=100
	KERNEL_SIZES=[3, 4, 5]
	LOG_INTERVAL=1
	LR=0.001
	MAX_NORM=3.0
	PREDICT=None
	SAVE_BEST=True
	SAVE_DIR=try1/2020-02-05_16-12-16
	SAVE_INTERVAL=500
	SHUFFLE=True
	SNAPSHOT=None
	STATIC=True
	TEST=False
	TEST_INTERVAL=100
Training sentences:  10400
Validation sentences:  2600


In [0]:
# model
cnn = CNN_Text(args, weight_matrix)
if args.snapshot is not None:
    print('\nLoading model from {}...'.format(args.snapshot))
    cnn.load_state_dict(torch.load(args.snapshot))

if args.cuda:
    torch.cuda.set_device(args.device)
    cnn = cnn.cuda()
        



In [0]:
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
# train or predict
# static
if args.predict is not None:
    label = predict(args.predict, cnn, text_field, label_field, args.cuda)
    print('\n[Text]  {}\n[Label] {}\n'.format(args.predict, label))
elif args.test:
    try:
        eval(test_iter, cnn, args) 
    except Exception as e:
        print("\nSorry. The test dataset doesn't  exist.\n")
else:
    print()
    try:
        val = train(train_loader, val_loader, cnn, args)
    except KeyboardInterrupt:
        print('\n' + '-' * 89)
        print('Exiting from training early')


On Cuda
epoch:  1


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1,   100] loss: 0.529
Training loss in epoch 1 is 0.498
Training accuracy in epoch 1 is 72.394
Training precision in epoch 1 is 26.119
Training recall in epoch 1 is 80.310
Training F1-score in epoch 1 is 39.418
Test loss in epoch 1 is 0.343
Test accuracy in epoch 1 is 85.231
Test precision in epoch 1 is 42.390
Test recall in epoch 1 is 89.003
Test F1-score in epoch 1 is 57.428

------------------------------------------------------------------------------------------

epoch:  2
[2,   100] loss: 0.359
Training loss in epoch 2 is 0.356
Training accuracy in epoch 2 is 83.000
Training precision in epoch 2 is 38.414
Training recall in epoch 2 is 86.242
Training F1-score in epoch 2 is 53.153
Test loss in epoch 2 is 0.308
Test accuracy in epoch 2 is 84.000
Test precision in epoch 2 is 40.516
Test recall in epoch 2 is 91.753
Test F1-score in epoch 2 is 56.211

------------------------------------------------------------------------------------------

epoch:  3
[3,   100] loss: 0.256
Training 

In [0]:
import pickle
with open('val_glove.pickle','wb') as f:
  pickle.dump(val,f)

In [0]:
torch.save(cnn, 'model_glove_2.pth')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type CNN_Text. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
